### source: https://www.onceupondata.com/2019/02/10/nn-collaborative-filtering/

In [24]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, Dropout, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

warnings.filterwarnings('ignore')
%matplotlib inline

## Loading in data

In [2]:
dataset = pd.read_csv('ratings.csv')

In [3]:
dataset.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [4]:
dataset.shape

(981756, 3)

In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

In [6]:
train.head()

,book_id,user_id,rating
341848,3423,4608,2
964349,9811,36373,5
645459,6485,2957,4
74960,750,42400,3
358670,3591,36886,5


In [7]:
test.head()

,book_id,user_id,rating
646451,6495,19643,5
614851,6175,8563,4
974393,9920,52110,3
21471,215,33864,5
272540,2728,16587,3


In [8]:
n_users = len(dataset.user_id.unique())
n_users

53424

In [9]:
n_books = len(dataset.book_id.unique())
n_books

10000

## Creating Neural Network
Neural Networks proved there effectivness for almost every machine learning problem as of now and they also perform exceptionally well for recommendation systems.

In [42]:
# define embeddingg size (similar for both users and books)
dim_embedddings = 50
bias = 1

# creating book embedding path
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books + 1, dim_embedddings, name="Book-Embedding")(book_input)
book_bias = Embedding(n_books + 1, bias, name="Book-Bias")(book_input)

# creating user embedding path
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users + 1, dim_embedddings, name="User-Embedding")(user_input)
user_bias = Embedding(n_users + 1, bias, name="User-Bias")(user_input)

matrix_product = Multiply()([book_embedding, user_embedding])
matrix_product = Dropout(0.2)(matrix_product)

input_terms = Concatenate()([matrix_product, user_bias, book_bias])
input_terms = Flatten()(input_terms)

# add fully-connected-layers
dense_1 = Dense(50, activation="relu", name = "Dense1")(input_terms)
dense_1 = Dropout(0.2)(dense_1)
dense_2 = Dense(20, activation="relu", name = "Dense2")(dense_1)
dense_2 = Dropout(0.2)(dense_2)
result = Dense(1, activation='relu', name='Activation')(dense_2)

# define model with 2 inputs and 1 output
model_mf = Model(inputs=[user_input, book_input], outputs=result)

# show model summary
model_mf.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Book-Input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
User-Input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
Book-Embedding (Embedding)      (None, 1, 50)        500050      Book-Input[0][0]                 
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 50)        2671250     User-Input[0][0]                 
____________________________________________________________________________________________

In [ ]:
from tensorflow.keras.models import load_model

training = True

if not training and os.path.exists('regression_model_mf.h5'):
    model_mf = load_model('regression_model_mf.h5')
else:
    # specify learning rate (or use the default)
    opt_adam = Adam(lr = 0.002)

    # compile model
    model_mf.compile(optimizer = opt_adam, loss = ['mse'], metrics = ['mean_absolute_error'])

    # fit model
    history_mf = model_mf.fit([train.user_id, train.book_id],
                              train.rating,
                              batch_size = 256,
                              validation_split = 0.005,
                              epochs = 5,
                              verbose = 1)
    model_mf.save('regression_model_mf.h5')
    plt.plot(history_mf.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

Train on 781476 samples, validate on 3928 samples
Epoch 1/5
781476/781476 [==============================] - 75s 96us/sample - loss: 0.2345 - mean_absolute_error: 0.3543 - val_loss: 0.8467 - val_mean_absolute_error: 0.6886
Epoch 2/5
781476/781476 [==============================] - 73s 94us/sample - loss: 0.2112 - mean_absolute_error: 0.3329 - val_loss: 0.8484 - val_mean_absolute_error: 0.6860
Epoch 3/5
781476/781476 [==============================] - 75s 96us/sample - loss: 0.1915 - mean_absolute_error: 0.3124 - val_loss: 0.8599 - val_mean_absolute_error: 0.6862
Epoch 4/5
381952/781476 [=============>................] - ETA: 38s - loss: 0.1683 - mean_absolute_error: 0.2874

In [44]:
model_mf.evaluate([test.user_id, test.book_id], test.rating)

196352/196352 [==============================] - 3s 17us/sample - loss: 0.8338 - mean_absolute_error: 0.6911


[0.8338121055604737, 0.6910621]

In [33]:
predictions = model_mf.predict([test.user_id.head(10), test.book_id.head(10)])

[print(predictions[i], test.rating.iloc[i]) for i in range(0,10)]

[4.907997] 5
[3.722946] 4
[4.2238035] 3
[3.964994] 5
[3.7542875] 3
[3.9106045] 3
[3.5167894] 3
[4.347043] 4
[4.151887] 3
[3.702667] 5


[None, None, None, None, None, None, None, None, None, None]

## Visualizing Embeddings
Embeddings are weights that are learned to represent some specific variable like books and user in our case and therefore we can not only use them to get good results on our problem but also to extract inside about our data.

In [ ]:
# Extract embeddings
book_em = model.get_layer('Book-Embedding')
book_em_weights = book_em.get_weights()[0]

In [ ]:
book_em_weights[:5]

In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns

pca = PCA(n_components=2)
pca_result = pca.fit_transform(book_em_weights)
sns.scatterplot(x=pca_result[:,0], y=pca_result[:,1])

In [ ]:
book_em_weights = book_em_weights / np.linalg.norm(book_em_weights, axis = 1).reshape((-1, 1))
book_em_weights[0][:10]
np.sum(np.square(book_em_weights[0]))

In [ ]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(book_em_weights)
sns.scatterplot(x=pca_result[:,0], y=pca_result[:,1])

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tnse_results = tsne.fit_transform(book_em_weights)

In [ ]:
sns.scatterplot(x=tnse_results[:,0], y=tnse_results[:,1])

## Making Recommendations

In [ ]:
# Creating dataset for making recommendations for the first user
book_data = np.array(list(set(dataset.book_id)))
book_data[:5]

In [ ]:
user = np.array([1 for i in range(len(book_data))])
user[:5]

In [ ]:
predictions = model.predict([user, book_data])

predictions = np.array([a[0] for a in predictions])

recommended_book_ids = (-predictions).argsort()[:5]

recommended_book_ids

In [ ]:
# print predicted scores
predictions[recommended_book_ids]

In [ ]:
books = pd.read_csv('books.csv')
books.head()

In [ ]:
books[books['id'].isin(recommended_book_ids)]